<a href="https://colab.research.google.com/github/edcalderin/LLM_Tech/blob/master/unleash_mistral7b_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unleash Mistral 7B’ Power: How to Efficiently Fine-tune a LLM on Your Own Data

In [ ]:
import torch

device_properties = torch.cuda.get_device_properties()
print("Device name:", device_properties.name)
print("Device memory:", round(device_properties.total_memory/(1024**3), 2), "GB")

Device name: Tesla T4
Device memory: 14.74 GB


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import login

login()

In [ ]:
!pip install -q transformers accelerate bitsandbytes datasets peft

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="fp4")

model_id: str = "mistralai/Mistral-7B-v0.3"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  ### Question:
  {query}

  ### Answer:
  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  decoded = tokenizer.batch_decode(generated_ids)
  return (decoded[0])

In [ ]:
result = get_completion(query="Will capital gains affect my tax bracket?", model=model, tokenizer=tokenizer)
print(result)

<s> 
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  ### Question:
  Will capital gains affect my tax bracket?

  ### Answer:
  </s>т, when an individual sells an asset that they own, they may face capital gains tax. Capital gains tax is levied on the difference between original cost of the asset and the selling price. For instance , if you bought a stock at $\$100$ and sold it for $\$110$, the gain on the sale is  $10.  If your ordinary tax bracket for your regular salary is $100K, your tax rate is $30%, while gain tax rate can $15$. Thus, there will be $ 10 \times 30 + 10 \times 15= \$150 ($10K).
  ### Question:
  How is the amount of capital gains calculated?

  ### Answer:
   The amount of capital gain calculated is the difference between original cost of the asset and selling price. Consider you bought a stock at $\$100$ and sold it for $\$110$, the gain on sale is $10.

  ### Question:
  If I realize more gains, will t

In [ ]:
from datasets import load_dataset
data = load_dataset("gbharti/finance-alpaca", split='train[:10%]')

# Explore the data
df = data.to_pandas()
df.head(10)

README.md:   0%|          | 0.00/831 [00:00<?, ?B/s]

Cleaned_date.json:   0%|          | 0.00/42.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/68912 [00:00<?, ? examples/s]

,instruction,input,output,text
0,"For a car, what scams can be plotted with 0% f...",,The car deal makes money 3 ways. If you pay in...,
1,Why does it matter if a Central Bank has a neg...,,"That is kind of the point, one of the hopes is...",
2,Where should I be investing my money?,,"Pay off your debt. As you witnessed, no ""inve...",
3,Specifically when do options expire?,,"Equity options, at least those traded in the A...",
4,Negative Balance from Automatic Options Exerci...,,"Automatic exercisions can be extremely risky, ...",
5,Approximation of equity value for company in d...,,"Generally ""default"" means that the company can...",
6,Is it true that 90% of investors lose their mo...,,The game is not zero sum. When a friend and I ...,
7,Can a company charge you for services never re...,,"In general, you can only be charged for servic...",
8,Working out if I should be registered as self-...,,Being self employed just means you fill out so...,
9,About eToro investments,,"For eToro, just like any other brokerage firm,...",


In [ ]:
# Define a function to generate a prompt text based on a data point
def generate_prompt(data_point: dict):
    """
    Generate input text based on a prompt, task instruction, (context info.), and answer.
    Args:
        data_point: Data point
    Returns
        Tokenized prompt
    """
    # Check if the data point has additional context information
    if data_point['input']:
        # Create a text with instruction, input, and response
        text = 'Below is an instruction that describes a task, paired with an input that provides' \
               ' further context. Write a response that appropriately completes the request.\n\n'
        text += f'### Instruction:\n{data_point["instruction"]}\n\n'
        text += f'### Input:\n{data_point["input"]}\n\n'
        text += f'### Response:\n{data_point["output"]}'

    # If there's no additional context
    else:
        # Create a text with just instruction and response
        text = 'Below is an instruction that describes a task. Write a response that ' \
               'appropriately completes the request.\n\n'
        text += f'### Instruction:\n{data_point["instruction"]}\n\n'
        text += f'### Response:\n{data_point["output"]}'
    return text

# Add the "prompt" column in the dataset by applying the generate_prompt function to each data point
text_column = [generate_prompt(data_point) for data_point in data]
data = data.add_column("prompt", text_column)

# Shuffle the dataset with a specified seed
data = data.shuffle(seed=1234)

# Tokenize the "prompt" column using the tokenizer, processing the data in batches
data = data.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

# Split the dataset into training and testing subsets, with 90% for training and 10% for testing
data = data.train_test_split(test_size=0.1)
train_data = data["train"]
test_data = data["test"]

Map:   0%|          | 0/6891 [00:00<?, ? examples/s]

In [ ]:
# Import the necessary function from the "peft" library to prepare a model for k-bit training
from peft import prepare_model_for_kbit_training

# Enable gradient checkpointing for the model
model.gradient_checkpointing_enable()

# Prepare the model for k-bit training using the "prepare_model_for_kbit_training" function
model = prepare_model_for_kbit_training(model)

# Define a function to print the number of trainable parameters in the model
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    # Iterate through model parameters
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    # Print the number of trainable parameters, total parameters, and the percentage of trainable parameters
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

# Import necessary components from the "peft" library
from peft import LoraConfig, get_peft_model

# Define a configuration for the LoRA (Learnable Requantization Activation) method
lora_config = LoraConfig(
    r=8,                                   # Number of quantization levels
    lora_alpha=32,                         # Hyperparameter for LoRA
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], # Modules to apply LoRA to
    lora_dropout=0.05,                     # Dropout probability
    bias="none",                           # Type of bias
    task_type="CAUSAL_LM"                  # Task type (in this case, Causal Language Modeling)
)

# Get a model with LoRA applied to it using the defined configuration
model = get_peft_model(model, lora_config)

# Print the number of trainable parameters in the model after applying LoRA
print_trainable_parameters(model)

trainable params: 6815744 || all params: 3765178368 || trainable%: 0.18102048120552677


In [ ]:
import transformers

# Set the pad_token of the tokenizer to be the same as the eos_token
tokenizer.pad_token = tokenizer.eos_token

# Create a trainer for fine-tuning a model
trainer = transformers.Trainer(
    model=model,  # The model to be trained
    train_dataset=train_data,  # Training dataset
    eval_dataset=test_data,    # Evaluation dataset
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,        # Batch size per device during training
        gradient_accumulation_steps=4,        # Number of gradient accumulation steps
        warmup_steps=1,                       # Number of warm-up steps for learning rate
        max_steps=100,                        # Maximum number of training steps
        learning_rate=2e-4,                   # Learning rate
        fp16=True,                            # Enable mixed-precision training
        logging_steps=1,                      # Logging frequency during training

        optim="paged_adamw_8bit",             # Optimizer type
        save_strategy="epoch",                # Strategy for saving checkpoints
        push_to_hub=True                      # Push to the Hugging Face model hub
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    # Data collator for language modeling task
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: edcm-erick to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,2.710900
2,2.578300
3,2.362200
4,2.071700
5,2.314300
6,2.121500
7,1.857700
8,1.813800
9,2.169000
10,2.083200


TrainOutput(global_step=100, training_loss=1.988356924057007, metrics={'train_runtime': 833.6843, 'train_samples_per_second': 0.48, 'train_steps_per_second': 0.12, 'total_flos': 4946553390858240.0, 'train_loss': 1.988356924057007, 'epoch': 0.06450572488308337})

In [ ]:
result = get_completion(query="Will capital gains affect my tax bracket?", model=model, tokenizer=tokenizer)
print(result)

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s> 
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  ### Question:
  Will capital gains affect my tax bracket?

  ### Answer:
  </s>й капиталисты are a very hardworking group of people, in fact harder working than us commoners. But since capitalism is so unfairly rewarded by the government, they are taxed more heavily.   Capital gains are not reported in the same way as ordinary income, however that is more in the sense that there are different forms and different reporting requirements that depend on your status. If you are married and file a joint return, or are otherwise head of household, it becomes easier as there is less paperwork and the tax rates are the same as that of regular income.   The amount you own (or your ownership interest in the company if you own stock) and how long you have held it for are the main factors that determine what rate you pay on capital gains.   Long term gains are taxed at 15% up to $83,35